In [29]:
from functools import partial
from typing import Callable

import arviz
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpyro
import pandas as pd
import seaborn as sns
import sklearn
from jax import Array
from numpyro import distributions as dist
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [30]:
iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(
    iris.data.astype(float), iris.target.astype(int), test_size=0.33, random_state=42
)

In [31]:
class Model:
    def __init__(self, x_dim=4, y_dim=3, h_dim=5):
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.h_dim = h_dim

    def __call__(self, x, y=None):
        """
        We need None for predictive
        """
        x_dim = self.x_dim
        y_dim = self.y_dim
        h_dim = self.h_dim
        # Number of observations
        n = x.shape[0]
        # standard deviation of Normals
        sd = 1  # EXERCISE: 100->1
        # Layer 1
        w1 = numpyro.sample("w1", dist.Normal(0, sd).expand((x_dim, h_dim)).to_event(2))
        b1 = numpyro.sample("b1", dist.Normal(0, sd).expand((h_dim,)).to_event(1))
        # Layer 2 # EXERCISE: added layer
        w2 = numpyro.sample("w2", dist.Normal(0, sd).expand((h_dim, h_dim)).to_event(2))
        b2 = numpyro.sample("b2", dist.Normal(0, sd).expand((h_dim,)).to_event(1))
        # Layer 3
        w3 = numpyro.sample("w3", dist.Normal(0, sd).expand((h_dim, y_dim)).to_event(2))
        b3 = numpyro.sample("b3", dist.Normal(0, sd).expand((y_dim,)).to_event(1))
        # NN
        h1 = jnp.tanh((x @ w1) + b1)
        h2 = jnp.tanh((h1 @ w2) + b2)  # EXERCISE: added layer
        logits = h2 @ w3 + b3
        # Save deterministc variable (logits) in trace
        numpyro.deterministic("logits", logits)
        # Categorical likelihood
        with numpyro.plate("labels", n):
            obs = numpyro.sample("obs", dist.Categorical(logits=logits), obs=y)
        return obs

In [32]:
kernel = numpyro.infer.NUTS(Model())
mcmc = numpyro.infer.MCMC(kernel, num_samples=1000, num_warmup=200, num_chains=4)
mcmc.run(jax.random.key(0), x=x_train, y=y_train)

/var/folders/pn/jqbv09dn34l3q3q4n4xwgf7h0000gn/T/ipykernel_76354/2397990501.py:2: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = numpyro.infer.MCMC(kernel, num_samples=1000, num_warmup=200, num_chains=4)
sample: 100%|██████████| 1200/1200 [00:07<00:00, 160.10it/s, 511 steps of size 8.91e-03. acc. prob=0.97]


In [33]:
arv = arviz.from_numpyro(mcmc)
arviz.summary(arv).describe()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
count,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000
mean,-0.002340,1.448810,-2.692094,2.732397,0.024965,0.018461,3898.935657,3470.597855,1.000509
std,2.141836,0.223968,2.364947,2.044342,0.007270,0.004809,1238.349594,581.353629,0.002434
min,-2.992000,0.818000,-6.353000,-0.134000,0.011000,0.013000,300.000000,845.000000,1.000000
25%,-1.657000,1.351000,-4.541000,1.236000,0.021000,0.016000,3882.000000,3490.000000,1.000000
50%,-0.089000,1.477000,-2.584000,2.128000,0.023000,0.018000,4389.000000,3709.000000,1.000000
75%,2.180000,1.572000,-0.276000,4.649000,0.027000,0.019000,4559.000000,3821.000000,1.000000
max,3.722000,1.885000,1.063000,6.703000,0.065000,0.046000,6039.000000,3974.000000,1.020000


In [42]:
acc_train = (mcmc.get_samples()["logits"].mean(0).argmax(1) == y_train).mean()
print(f"Training accuracy: {acc_train:.1%}")

Training accuracy: 98.0%


In [50]:
pred_logits = numpyro.infer.Predictive(Model(), mcmc.get_samples())(
    jax.random.key(0), x_test
)["logits"]
acc_test = (pred_logits.mean(0).argmax(1) == y_test).mean()
print(f"Test accuracy: {acc_test:.1%}")

Test accuracy: 98.0%
